In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import warnings

warnings.simplefilter('ignore')

# RID OSPAR Reporting Template

**Note:** Despite the [changes to the programme](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/programme_changes_2017-18.ipynb) for 2017 to 2020, we have decided to continue to submit data to OSPAR using the old divisions (see e-mail from Øyvind received 15.10.2018 at 15.51). The code in this notebook therefore uses the column named `'old_rid_code'` to allocate fluxes to the correct regions.

The final step in the annual RID reporting is to complete and submit an overall summary template to OSPAR. A blank template and guideline documentation were provided by Eva (see e-mail received 04/09/2017 at 13.07) and the 2015 template created by Tore is here:

K:\Avdeling\Vass\316_Miljøinformatikk\Prosjekter\RID\2016\OSPAR\Norway_2015.xlsx

**Note:** Tore previously calculated "upper" load estimates by setting LOD values equal to the LOD itself, and "lower" estimates by setting LOD values equal to zero. This year we have handled LOD values more explicity, by calculating a single estimate as described [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/rid_data_exploration.ipynb#2.3.-Calculate-loads). We therefore only need to fill-in entires for the "mean" rows.

Most of the information required for the template has already been reported in other tables, especially the following:

 * *Table_3_2016.docx* <br><br>
 
 * *loads_and_flows_all_sites_2016.csv* <br><br>
 
 * *concs_and_flows_rid_11-36_2016.csv*

However, some information is not available from these tables, and in previous years Tore has simply left these cells blank. The code here does the same, although in principle we could fill-in the whole thing with a bit more work.

## 1. Get summary data

The notebook [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/summary_table_2016.ipynb) calculates summary values for Table 3. Rather than reading these values from Word, it is easier to repeat the code to summarise the raw data again. This is the data that needs writing to the OSPAR template.

### 1.1. Monitored areas

In [2]:
# Set year of interest
year = 2019

In [3]:
# Read data
in_csv = r'../../../Results/Loads_CSVs/loads_and_flows_all_sites_%s.csv' % year
mon_df = pd.read_csv(in_csv)
del mon_df['new_rid_group']

# Group by OSPAR region
mon_df1 = mon_df.groupby(['ospar_region', 'old_rid_group']).sum()

# Totals for Norway
mon_df2 = mon_df.groupby('old_rid_group').sum().reset_index()
mon_df2['ospar_region'] = 'NORWAY'
mon_df2.set_index(['ospar_region', 'old_rid_group'], inplace=True)

# Combine
mon_df = pd.concat([mon_df1, mon_df2], axis=0)

# Cols of interest
cols = [i for i in mon_df.columns if i.split('_')[1] != 'Est']
mon_df = mon_df[cols]
del mon_df['station_id'], mon_df['mean_q_1000m3/day']

# Convert units
mon_df['Hg_kg'] = mon_df['Hg_kg']/1000. # kg to tonnes
mon_df['NH4-N_tonnes'] = mon_df['NH4-N_tonnes']/1000. # tonnes to ktonnes
mon_df['NO3-N_tonnes'] = mon_df['NO3-N_tonnes']/1000. # tonnes to ktonnes
mon_df['TOTN_tonnes'] = mon_df['TOTN_tonnes']/1000. # tonnes to ktonnes
mon_df['TOTP_tonnes'] = mon_df['TOTP_tonnes']/1000. # tonnes to ktonnes
mon_df['PO4-P_tonnes'] = mon_df['PO4-P_tonnes']/1000. # tonnes to ktonnes
mon_df['SPM_tonnes'] = mon_df['SPM_tonnes']/1000. # tonnes to ktonnes

# Units are correct, so remove
mon_df.columns = [i.split('_')[0] for i in mon_df.columns]

mon_df.round(0)

Ag   As   Cd    Cr     Cu   Hg  NH4-N  \
ospar_region        old_rid_group                                           
LOFOTEN-BARENTS SEA rid_108        0.0  1.0  0.0   1.0    6.0  0.0    0.0   
                    rid_11         0.0  0.0  0.0   0.0    1.0  0.0    0.0   
                    rid_36         0.0  2.0  0.0   6.0   60.0  0.0    0.0   
NORTH SEA           rid_108        0.0  1.0  0.0   1.0    9.0  0.0    0.0   
                    rid_11         0.0  0.0  0.0   0.0    1.0  0.0    0.0   
                    rid_36         0.0  2.0  0.0   2.0   11.0  0.0    0.0   
NORWEGIAN SEA2      rid_108        0.0  1.0  0.0   1.0    6.0  0.0    0.0   
                    rid_11         0.0  1.0  0.0   1.0   13.0  0.0    0.0   
                    rid_36         0.0  3.0  0.0   9.0   27.0  0.0    0.0   
SKAGERAK            rid_108        0.0  1.0  0.0   2.0    4.0  0.0    0.0   
                    rid_11         0.0  8.0  1.0   9.0   60.0  0.0    1.0   
                    rid_36         0.0  2.0  0.0   1.0    8.0  0.0    0.0   
NORWAY              rid_108        0.0  5.0  0.0   4.0   26.0  0.0    0.0   
                    rid_11         0.0  9.0  1.0  10.0   76.0  0.0    1.0   
                    rid_36         0.0  9.0  1.0  18.0  106.0  0.0    1.0   

                                   NO3-N     Ni  PO4-P    Pb    SPM      SiO2  \
ospar_region        old_rid_group                                               
LOFOTEN-BARENTS SEA rid_108          0.0    5.0    0.0   0.0   11.0   25874.0   
                    rid_11           0.0    1.0    0.0   0.0   13.0   12570.0   
                    rid_36           1.0   98.0    0.0   1.0   62.0   91798.0   
NORTH SEA           rid_108          3.0    6.0    0.0   3.0   23.0   25682.0   
                    rid_11           0.0    1.0    0.0   0.0    5.0    3005.0   
                    rid_36           3.0    4.0    0.0   4.0   39.0   31134.0   
NORWEGIAN SEA2      rid_108          1.0    4.0    0.0   1.0   24.0   20463.0   
                    rid_11           1.0    3.0    0.0   0.0    7.0   13276.0   
                    rid_36           3.0   20.0    0.0   2.0  108.0   72458.0   
SKAGERAK            rid_108          2.0    3.0    0.0   2.0   17.0   13084.0   
                    rid_11          17.0   30.0    0.0  10.0  280.0  166188.0   
                    rid_36           2.0    4.0    0.0   3.0   18.0   24800.0   
NORWAY              rid_108          6.0   19.0    0.0   6.0   75.0   85103.0   
                    rid_11          18.0   34.0    0.0  10.0  304.0  195039.0   
                    rid_36           8.0  126.0    0.0  10.0  227.0  220190.0   

                                        TOC  TOTN  TOTP     Zn  
ospar_region        old_rid_group                               
LOFOTEN-BARENTS SEA rid_108         27810.0   1.0   0.0    7.0  
                    rid_11           9657.0   0.0   0.0    0.0  
                    rid_36          53747.0   3.0   0.0   59.0  
NORTH SEA           rid_108         25868.0   5.0   0.0   27.0  
                    rid_11           5059.0   1.0   0.0    3.0  
                    rid_36          39048.0   6.0   0.0   51.0  
NORWEGIAN SEA2      rid_108         21761.0   2.0   0.0    9.0  
                    rid_11          14000.0   1.0   0.0   27.0  
                    rid_36          74337.0   7.0   0.0   66.0  
SKAGERAK            rid_108         21346.0   3.0   0.0   28.0  
                    rid_11         215526.0  26.0   1.0  218.0  
                    rid_36          58387.0   4.0   0.0   44.0  
NORWAY              rid_108         96785.0  11.0   0.0   71.0  
                    rid_11         244241.0  28.0   1.0  248.0  
                    rid_36         225519.0  20.0   1.0  220.0

### 1.2. Unmonitored areas

In [4]:
# Read data
in_csv = r'../../../Results/Unmon_loads/unmon_loads_%s.csv' % year
umon_df = pd.read_csv(in_csv, index_col=0)

# Rename cols
umon_df.columns = [i.replace('RENSEANLEGG', 'sew') for i in umon_df.columns]
umon_df.columns = [i.replace('INDUSTRI', 'ind') for i in umon_df.columns]
umon_df.columns = [i.replace('_tonn', '') for i in umon_df.columns]
umon_df.columns = [i.replace('AQUAKULTUR', 'fish') for i in umon_df.columns]

# Convert Hg to kgs
umon_df['sew_Hg'] = umon_df['sew_Hg']*1000
umon_df['ind_Hg'] = umon_df['ind_Hg']*1000

umon_df.round(0)

,flow,sew_n,sew_p,ind_n,ind_p,fish_n,fish_p,diff_n,diff_p,sew_po4,...,sew_S.P.M.,sew_As,sew_Pb,sew_Cd,sew_Cu,sew_Zn,sew_Ni,sew_Cr,sew_Hg,fish_Cu
ospar_region,,,,,,,,,,,,,,,,,,,,,
NORWAY,425297.0,13310.0,1104.0,2476.0,231.0,63365.0,10925.0,36780.0,725.0,662.0,...,15640.0,0.0,0.0,0.0,6.0,15.0,2.0,0.0,3.0,1217.0
LOFOTEN-BARENTS SEA,127196.0,1543.0,180.0,27.0,2.0,15664.0,2712.0,5953.0,111.0,108.0,...,2506.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,302.0
NORTH SEA,134234.0,3534.0,441.0,470.0,83.0,20361.0,3501.0,14173.0,217.0,265.0,...,6169.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,389.0
NORWEGIAN SEA2,150979.0,2795.0,367.0,1051.0,105.0,27283.0,4703.0,13422.0,302.0,220.0,...,4565.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,525.0
SKAGERAK,12888.0,5438.0,116.0,927.0,41.0,56.0,10.0,3232.0,96.0,70.0,...,2400.0,0.0,0.0,0.0,4.0,10.0,1.0,0.0,1.0,1.0


### 1.3. Loads for 11 main rivers

In [5]:
# Read data
in_csv = r'../../../Results/Loads_CSVs/loads_and_flows_all_sites_%s.csv' % year
rid11_df = pd.read_csv(in_csv, index_col=0)
del rid11_df['new_rid_group']

# Get data for RID11
rid11_df = rid11_df.query('old_rid_group == "rid_11"')

# Tidy
del rid11_df['station_code'], rid11_df['station_name']
del rid11_df['old_rid_group'], rid11_df['ospar_region']
del rid11_df['mean_q_1000m3/day']

cols = [i for i in rid11_df.columns if i.split('_')[1] != 'Est']
rid11_df = rid11_df[cols]

# Convert units
rid11_df['Hg_kg'] = rid11_df['Hg_kg']/1000. # kg to tonnes
rid11_df['NH4-N_tonnes'] = rid11_df['NH4-N_tonnes']/1000. # tonnes to ktonnes
rid11_df['NO3-N_tonnes'] = rid11_df['NO3-N_tonnes']/1000. # tonnes to ktonnes
rid11_df['TOTN_tonnes'] = rid11_df['TOTN_tonnes']/1000. # tonnes to ktonnes
rid11_df['TOTP_tonnes'] = rid11_df['TOTP_tonnes']/1000. # tonnes to ktonnes
rid11_df['PO4-P_tonnes'] = rid11_df['PO4-P_tonnes']/1000. # tonnes to ktonnes
rid11_df['SPM_tonnes'] = rid11_df['SPM_tonnes']/1000. # tonnes to ktonnes

# Tidy cols
rid11_df.columns = [i.split('_')[0] for i in rid11_df.columns]

rid11_df.head()

,Ag,As,Cd,Cr,Cu,Hg,NH4-N,NO3-N,Ni,PO4-P,Pb,SPM,SiO2,TOC,TOTN,TOTP,Zn
station_id,,,,,,,,,,,,,,,,,
29612,0.011305,1.434508,0.085463,1.106697,6.952613,0.031962,0.104496,3.554804,4.744528,0.021984,0.719532,16.797207,32190.308688,39925.217256,5.080432,0.058825,19.581212
29779,0.009226,0.290131,0.000000,0.490114,1.205842,0.001539,0.002836,0.081307,0.639669,0.005679,0.034361,12.836286,12569.725807,9656.521305,0.358547,0.016590,0.376810
29821,0.005187,0.211410,0.012018,0.121396,0.903471,0.003849,0.005570,0.224318,0.712928,0.002511,0.167388,1.977466,2569.100624,3476.205353,0.407182,0.009454,1.946170
29782,0.000000,0.495405,0.009239,0.320848,1.469240,0.003925,0.007519,0.180850,1.370321,0.005885,0.115531,4.429221,6947.073303,7107.549670,0.446387,0.017857,1.603942
36225,0.000295,0.023209,0.002153,0.024893,0.204626,0.000082,0.000898,0.046567,0.057100,0.006834,0.035243,4.841364,349.324628,557.963453,0.077028,0.008333,0.620023


## 2. Fill-in template

A copy of the 2018 template from Eva is here:

    ./Results/OSPAR_Template/01_OSPAR_Norway_2018.xlsx

In [6]:
# Copy of template to update
temp_path = r'../../../Results/OSPAR_Template/01_OSPAR_Norway_%s.xlsx' % year

In [7]:
def update_spreadsheet_point_sources(xlsx, sheet, pars, src, df):
    """ Update the OSPAR template for point source data.
    
    Args:
        xslx:  Str. Path to Excel template
        sheet: Str. Sheet name to update
        pars:  List. Parameter names in template to fill-in
        src:   Str. Type of input ('sew', 'ind', 'fish')
        df:    Dataframe. Values to fill-in
        
    Returns:
        None. The template is updated and saved.
    """
    import pandas as pd
    from openpyxl import load_workbook
    
    # Map Excel headings to df cols
    par_dict = {'SPM':'S.P.M.',
                'TOC':'TOC',
                'PO4-P':'po4',
                'P-Total':'p',
                'NO3-N':'no3', 
                'NH4-N':'nh4',
                'N-Total':'n',
                'As':'As',
                'Pb':'Pb',
                'Cd':'Cd', 
                'Cu':'Cu',
                'Zn':'Zn',
                'Ni':'Ni',
                'Total Cr':'Cr',
                'Hg':'Hg'}

    # Map template names to df names
    names_dict = {'Norwegian Sea (NO)':'NORWEGIAN SEA2',
                  'Barents Sea (NO)':'LOFOTEN-BARENTS SEA',
                  'Skagerrak (NO)':'SKAGERAK',
                  'North Sea (NO)':'NORTH SEA',
                  'Norway Total':'NORWAY'}

    # Open new file and get sheet
    wb = load_workbook(filename=xlsx)
    ws = wb[sheet]

    # Get row numbers
    row_dict = {}
    for item in ws['B12':'B%s' % ws.max_row]:
        # Get cell properties
        cell = item[0]
        name = cell.value
        row = cell.row
        row_dict[name] = row

    # Get col numbers
    col_dict = {}
    for cell in ws['E9':'AK9'][0]:
        # Get cell properties
        par = cell.value
        col = cell.column
        col_dict[par] = col

    # Update spreadsheet
    for reg in names_dict.keys():
        for par in pars:
            # Get value from df
            val = df.loc[names_dict[reg], '%s_%s' % (src, par_dict[par])]

            # Get cell co-ords
            row = row_dict[reg] + 2
            col = col_dict[par]
           
            # Write value
            ws.cell(row=row, column=col).value = val

    # Save
    wb.save(xlsx)

### 2.1. Sheet 5a: Sewage effluents

Note these are the values from Table 3 for **unmonitored areas** and not the total sewage inputs for each of the OSPAR areas. This is the same as what Tore reported previously.

In [8]:
# Get Sewage data
cols = [i for i in umon_df.columns if i.split('_')[0]=='sew']
sew_df = umon_df[cols].copy()

# Convert units
sew_df['sew_Hg'] = sew_df['sew_Hg']/1000. # kg to tonnes
sew_df['sew_nh4'] = sew_df['sew_nh4']/1000. # tonnes to ktonnes
sew_df['sew_no3'] = sew_df['sew_no3']/1000. # tonnes to ktonnes
sew_df['sew_n'] = sew_df['sew_n']/1000. # tonnes to ktonnes
sew_df['sew_po4'] = sew_df['sew_po4']/1000. # tonnes to ktonnes
sew_df['sew_p'] = sew_df['sew_p']/1000. # tonnes to ktonnes
sew_df['sew_S.P.M.'] = sew_df['sew_S.P.M.']/1000. # tonnes to ktonnes

sew_df

,sew_n,sew_p,sew_po4,sew_no3,sew_nh4,sew_S.P.M.,sew_As,sew_Pb,sew_Cd,sew_Cu,sew_Zn,sew_Ni,sew_Cr,sew_Hg
ospar_region,,,,,,,,,,,,,,
NORWAY,13.310128,1.103756,0.662254,0.665506,9.982596,15.640013,0.315579,0.274782,0.017832,5.721748,15.271116,1.824993,0.486468,0.002516
LOFOTEN-BARENTS SEA,1.542726,0.179667,0.107800,0.077136,1.157044,2.506221,0.007278,0.006965,0.000307,0.142696,0.401504,0.055732,0.116209,0.000160
NORTH SEA,3.533976,0.441060,0.264636,0.176699,2.650482,6.168894,0.093861,0.060930,0.003304,0.707030,2.533159,0.304204,0.050879,0.000389
NORWEGIAN SEA2,2.795488,0.366835,0.220101,0.139774,2.096616,4.564504,0.045849,0.036585,0.001992,1.097526,1.966538,0.159613,0.094464,0.000604
SKAGERAK,5.437938,0.116194,0.069717,0.271897,4.078454,2.400394,0.168591,0.170302,0.012229,3.774496,10.369915,1.305444,0.224916,0.001363


In [9]:
# Update sheet 5a
pars = ['Ni', 'Pb', 'NH4-N', 'Total Cr', 'NO3-N', 
        'Zn', 'As', 'Cd', 'P-Total', 'SPM', 'PO4-P', 
        'N-Total', 'Hg', 'Cu']

update_spreadsheet_point_sources(temp_path, '5a', pars, 'sew', sew_df)

### 2.2. Sheet 5b: Industrial effluents

In [10]:
# Get industrial data
cols = [i for i in umon_df.columns if i.split('_')[0]=='ind']
ind_df = umon_df[cols].copy()

# Convert units
ind_df['ind_Hg'] = ind_df['ind_Hg']/1000. # kg to tonnes
ind_df['ind_nh4'] = ind_df['ind_nh4']/1000. # tonnes to ktonnes
ind_df['ind_no3'] = ind_df['ind_no3']/1000. # tonnes to ktonnes
ind_df['ind_n'] = ind_df['ind_n']/1000. # tonnes to ktonnes
ind_df['ind_po4'] = ind_df['ind_po4']/1000. # tonnes to ktonnes
ind_df['ind_p'] = ind_df['ind_p']/1000. # tonnes to ktonnes
ind_df['ind_S.P.M.'] = ind_df['ind_S.P.M.']/1000. # tonnes to ktonnes

ind_df

,ind_n,ind_p,ind_po4,ind_no3,ind_nh4,ind_S.P.M.,ind_TOC,ind_As,ind_Pb,ind_Cd,ind_Cu,ind_Zn,ind_Ni,ind_Cr,ind_Hg
ospar_region,,,,,,,,,,,,,,,
NORWAY,2.475620,0.231055,0.138633,0.123781,1.856715,456.396912,1108.6823,2.307178,0.662853,0.078838,6.597055,9.950438,5.517691,1.124483,0.006689
LOFOTEN-BARENTS SEA,0.027312,0.001550,0.000930,0.001366,0.020484,242.532814,117.3264,0.000246,0.000482,0.000014,0.000201,0.001181,0.000840,0.000806,0.000004
NORTH SEA,0.470058,0.083131,0.049878,0.023503,0.352544,7.720511,796.6190,2.103018,0.488363,0.041900,0.580482,3.483454,3.208283,0.415410,0.002269
NORWEGIAN SEA2,1.051297,0.105015,0.063009,0.052565,0.788473,205.012687,115.6468,0.014715,0.036601,0.013648,0.798370,1.626187,0.141550,0.040129,0.000356
SKAGERAK,0.926953,0.041359,0.024815,0.046348,0.695215,1.130901,79.0901,0.189199,0.137406,0.023277,5.218003,4.839617,2.167018,0.668137,0.004060


In [11]:
# Update sheet 5b
pars = ['Ni', 'Pb', 'NH4-N', 'Total Cr', 'NO3-N', 
        'Zn', 'As', 'Cd', 'P-Total', 'SPM', 'PO4-P', 
        'N-Total', 'Hg', 'Cu', 'TOC']

update_spreadsheet_point_sources(temp_path, '5b', pars, 'ind', ind_df)

### 2.3. Sheet 5c: Aquaculture discharges

In [12]:
# Get fish data
cols = [i for i in umon_df.columns if i.split('_')[0]=='fish']
fish_df = umon_df[cols].copy()

# Convert units
fish_df['fish_nh4'] = fish_df['fish_nh4']/1000. # tonnes to ktonnes
fish_df['fish_no3'] = fish_df['fish_no3']/1000. # tonnes to ktonnes
fish_df['fish_n'] = fish_df['fish_n']/1000. # tonnes to ktonnes
fish_df['fish_po4'] = fish_df['fish_po4']/1000. # tonnes to ktonnes
fish_df['fish_p'] = fish_df['fish_p']/1000. # tonnes to ktonnes

fish_df

,fish_n,fish_p,fish_po4,fish_no3,fish_nh4,fish_Cu
ospar_region,,,,,,
NORWAY,63.364837,10.925302,7.538458,6.970132,50.691870,1216.712752
LOFOTEN-BARENTS SEA,15.664122,2.712265,1.871463,1.723053,12.531297,301.653881
NORTH SEA,20.361371,3.500562,2.415388,2.239751,16.289097,389.327100
NORWEGIAN SEA2,27.283301,4.702940,3.245028,3.001163,21.826641,524.671349
SKAGERAK,0.056044,0.009535,0.006579,0.006165,0.044835,1.060423


In [13]:
# Update sheet 5c
pars = ['NH4-N', 'NO3-N', 'P-Total', 'PO4-P', 'N-Total', 'Cu']

update_spreadsheet_point_sources(temp_path, '5c', pars, 'fish', fish_df)

### 2.4. Sheet 5d: Other discharges

This sheet is left blank

### 2.5. Sheet 5e: Total direct discharges

The sum of sewage, industrial and fish-farm discharges.

In [14]:
# Combine sew, ind and fish, then aggregate
for df in [sew_df, ind_df, fish_df]:
    df.reset_index(inplace=True)
    df.columns = [i.split('_')[1] for i in df.columns]

td_df = pd.concat([sew_df, ind_df, fish_df], axis=0, sort=True)
td_df = td_df.groupby('region').sum()

td_df.columns = ['td_'+i for i in td_df.columns]

td_df

,td_As,td_Cd,td_Cr,td_Cu,td_Hg,td_Ni,td_Pb,td_S.P.M.,td_TOC,td_Zn,td_n,td_nh4,td_no3,td_p,td_po4
region,,,,,,,,,,,,,,,
LOFOTEN-BARENTS SEA,0.007524,0.000321,0.117015,301.796778,0.000164,0.056572,0.007447,245.039035,117.3264,0.402685,17.234160,13.708826,1.801555,2.893482,1.980193
NORTH SEA,2.196879,0.045204,0.466289,390.614612,0.002658,3.512487,0.549293,13.889405,796.6190,6.016613,24.365405,19.292122,2.439952,4.024753,2.729903
NORWAY,2.622757,0.096670,1.610951,1229.031556,0.009205,7.342684,0.937635,472.036926,1108.6823,25.221554,79.150586,62.531181,7.759420,12.260112,8.339345
NORWEGIAN SEA2,0.060564,0.015640,0.134593,526.567245,0.000960,0.301163,0.073186,209.577191,115.6468,3.592725,31.130086,24.711730,3.193502,5.174790,3.528138
SKAGERAK,0.357790,0.035506,0.893053,10.052921,0.005423,3.472462,0.307708,3.531294,79.0901,15.209532,6.420935,4.818504,0.324409,0.167088,0.101111


In [15]:
# Update sheet 5e
pars = ['Ni', 'Pb', 'NH4-N', 'Total Cr', 'NO3-N', 
        'Zn', 'As', 'Cd', 'P-Total', 'SPM', 'PO4-P', 
        'N-Total', 'Hg', 'Cu', 'TOC']

update_spreadsheet_point_sources(temp_path, '5e', pars, 'td', td_df)

### 2.6. Sheet 6a: Monitored rivers

**Does "Inner Oslofjord" in the template correspond to "Alna"?** If so, I can fill-in one additional row in this table.

In [16]:
mon_df.reset_index(inplace=True)
tot_df = mon_df.groupby('ospar_region').sum()
trib_df = mon_df[mon_df['old_rid_group']!='rid_11'].groupby('ospar_region').sum()

In [17]:
def update_spreadsheet_monitored_rivers(xlsx, sheet, pars, df_dict):
    """ Update the OSPAR template for monitored rivers.
    
    Args:
        xslx:    Str. Path to Excel template
        sheet:   Str. Sheet name to update
        pars:    List. Parameter names in template to fill-in
        df_dict: Dict. {'tot':tot_df, 'trib':trib_df, 'main':rid11_df}
                 Values to fill-in
        
    Returns:
        None. The template is updated and saved.
    """
    import pandas as pd
    from openpyxl import load_workbook

    # Map Excel headings to df cols
    par_dict = {'SPM':'SPM',
                'TOC':'TOC',
                'PO4-P':'PO4-P',
                'P-Total':'TOTP',
                'NO3-N':'NO3-N', 
                'NH4-N':'NH4-N',
                'N-Total':'TOTN',
                'As':'As',
                'Pb':'Pb',
                'Cd':'Cd', 
                'Cu':'Cu',
                'Zn':'Zn',
                'Ni':'Ni',
                'Total Cr':'Cr',
                'Hg':'Hg'}
  
    # Map template names to df names and rows
    names_dict = {'Norwegian Sea (NO)':('NORWEGIAN SEA2', 'tot'),
                  'Barents Sea (NO)':('LOFOTEN-BARENTS SEA', 'tot'),
                  'Skagerrak (NO)':('SKAGERAK', 'tot'),
                  'North Sea (NO)':('NORTH SEA', 'tot'),
                  'Norway Total':('NORWAY', 'tot'),
                  'Tributary Rivers - Norwegian Sea':('NORWEGIAN SEA2', 'trib'),
                  'Tributary Rivers - Barents Sea':('LOFOTEN-BARENTS SEA', 'trib'),
                  'Tributary Rivers - Skagerak':('SKAGERAK', 'trib'),
                  'Tributary Rivers - North Sea':('NORTH SEA', 'trib'),
                  'Orkla':(29778, 'main'),
                  'Vefsna':(29782, 'main'),
                  'Alta':(29779, 'main'),
                  'Glomma':(29617, 'main'),
                  'Drammenselva':(29612, 'main'),
                  u'Numedalslågen':(29615, 'main'),
                  'Skienselva':(29613, 'main'),
                  'Otra':(29614, 'main'),
                  'Orreelva':(29783, 'main'),
                  'Vosso':(29821, 'main')}

    # Open new file and get sheet
    wb = load_workbook(filename=xlsx)
    ws = wb[sheet]

    # Get row numbers
    row_dict = {}
    for item in ws['B12':'B%s' % ws.max_row]:
        # Get cell properties
        cell = item[0]
        name = cell.value
        row = cell.row
        row_dict[name] = row

    # Get col numbers
    col_dict = {}
    for cell in ws['E9':'AK9'][0]:
        # Get cell properties
        par = cell.value
        col = cell.column
        col_dict[par] = col

    # Update spreadsheet
    for reg in names_dict.keys():
        df_idx, df_name = names_dict[reg]
        
        # Get df 
        df = df_dict[df_name]
        
        for par in pars:
            # Get value from df
            val = df.loc[df_idx, par_dict[par]]

            # Get cell co-ords
            row = row_dict[reg] + 2
            col = col_dict[par]

            # Write value
            ws.cell(row=row, column=col).value = val

    # Save
    wb.save(xlsx)

In [18]:
# Update sheet 6a
pars = ['Ni', 'Pb', 'NH4-N', 'Total Cr', 'NO3-N', 
        'Zn', 'As', 'Cd', 'P-Total', 'SPM', 'PO4-P', 
        'N-Total', 'Hg', 'Cu', 'TOC']
df_dict = {'tot':tot_df,
           'main':rid11_df,
           'trib':trib_df}

update_spreadsheet_monitored_rivers(temp_path, '6a', pars, df_dict)

### 2.7. Sheet 6b: Unmonitored areas

In [19]:
# Get diff data
cols = [i for i in umon_df.columns if i.split('_')[0]=='diff']
diff_df = umon_df[cols].copy()

# Convert units
diff_df['diff_nh4'] = diff_df['diff_nh4']/1000. # tonnes to ktonnes
diff_df['diff_no3'] = diff_df['diff_no3']/1000. # tonnes to ktonnes
diff_df['diff_n'] = diff_df['diff_n']/1000. # tonnes to ktonnes
diff_df['diff_po4'] = diff_df['diff_po4']/1000. # tonnes to ktonnes
diff_df['diff_p'] = diff_df['diff_p']/1000. # tonnes to ktonnes

diff_df

,diff_n,diff_p,diff_po4,diff_no3,diff_nh4
ospar_region,,,,,
NORWAY,36.779562,0.725093,0.178373,22.987226,2.022876
LOFOTEN-BARENTS SEA,5.952705,0.110915,0.027285,3.720441,0.327399
NORTH SEA,14.172717,0.216762,0.053323,8.857948,0.779499
NORWEGIAN SEA2,13.421980,0.301797,0.074242,8.388738,0.738209
SKAGERAK,3.232160,0.095620,0.023522,2.020100,0.177769


In [20]:
# Update sheet 6b
pars = ['NH4-N', 'NO3-N', 'P-Total', 'PO4-P', 'N-Total']

update_spreadsheet_point_sources(temp_path, '6b', pars, 'diff', diff_df)

### 2.8. Sheet 6c: Total inputs

**Note:** See e-mail from Csilla received 07/11/2017 at 13.43. This table should **not** include "point" discharges (`td_df`) - it's just the sum of tables 6a and 6b.

In [21]:
# Rename cols in diff_df
col_map = {'diff_n':'TOTN',
           'diff_p':'TOTP',
           'diff_po4':'PO4-P',
           'diff_no3':'NO3-N',
           'diff_nh4':'NH4-N'}
diff_df.columns = [col_map[i] for i in diff_df.columns]

# Add to total_df
for col in diff_df.columns:
    tot_df[col] = tot_df[col] + diff_df[col]

# Update sheet 6c
pars = ['Ni', 'Pb', 'NH4-N', 'Total Cr', 'NO3-N', 
        'Zn', 'As', 'Cd', 'P-Total', 'SPM', 'PO4-P', 
        'N-Total', 'Hg', 'Cu', 'TOC']

df_dict = {'tot':tot_df,
           'main':rid11_df,
           'trib':trib_df}

update_spreadsheet_monitored_rivers(temp_path, '6c', pars, df_dict)

The following two cells are not longer used as I originally misunderstood what table 6c represents.

In [22]:
## Standardise col names
#diff_df.columns = [i.split('_')[1] for i in diff_df.columns]
#td_df.columns = [i.split('_')[1] for i in td_df.columns]
#td_df.index.name = 'ospar_region'
#
#col_dict = {'SPM':'S.P.M.',
#            'TOTN':'n',
#            'NH4-N':'nh4',
#            'NO3-N':'no3',
#            'TOTP':'p',
#            'PO4-P':'po4'}
#for col in col_dict.keys():
#    new_col = col_dict[col]
#    tot_df[new_col] = tot_df[col]
#    del tot_df[col]
#    
## Reset index
#diff_df.reset_index(inplace=True)
#tot_df.reset_index(inplace=True)
#td_df.reset_index(inplace=True)
#
## Concat and aggregate
##tot_df = pd.concat([diff_df, td_df, tot_df], axis=0).groupby('ospar_region').sum() # See comment above
#tot_df = pd.concat([diff_df, tot_df], axis=0).groupby('ospar_region').sum()
#
## Rename cols
#tot_df.columns = ['tot_'+i for i in tot_df.columns]
#
#tot_df

In [23]:
## Update sheet 6c
#pars = ['Ni', 'Pb', 'NH4-N', 'Total Cr', 'NO3-N', 
#        'Zn', 'As', 'Cd', 'P-Total', 'SPM', 'PO4-P', 
#        'N-Total', 'Hg', 'Cu', 'TOC']
#
#update_spreadsheet_point_sources(temp_path, '6c', pars, 'tot', tot_df)

### 2.9. Sheet 7: Concentrations

In [24]:
# Read data
in_csv = r'../../../Results/Loads_CSVs/concs_and_flows_rid_20_%s.csv' % year
conc_df = pd.read_csv(in_csv, index_col = 0, encoding='utf-8')

# Convert units
conc_df['Hg_ng/l'] = conc_df['Hg_ng/l'] / 1000 # ng to ug
conc_df[u'NH4-N_µg/l N'] = conc_df[u'NH4-N_µg/l N'] / 1000 # ug to mg
conc_df[u'NO3-N_µg/l N'] = conc_df[u'NO3-N_µg/l N'] / 1000 # ug to mg
conc_df[u'TOTN_µg/l N'] = conc_df[u'TOTN_µg/l N'] / 1000 # ug to mg
conc_df[u'TOTP_µg/l P'] = conc_df[u'TOTP_µg/l P'] / 1000 # ug to mg
conc_df[u'PO4-P_µg/l P'] = conc_df[u'PO4-P_µg/l P'] / 1000 # ug to mg
conc_df['TOC_mg C/l'] = conc_df['TOC_mg C/l'] * 1000 # mg to ug

# Get flags
cols = [i for i in conc_df.columns if i.split('_')[1]=='flag']
lod_df = conc_df[cols]
lod_df.columns = [i.split('_')[0] for i in lod_df.columns]

# Get vals
cols = [i for i in conc_df.columns if ((i.split('_')[0] in lod_df.columns) 
                                       and (i.split('_')[1] != 'flag'))]
conc_df = conc_df[cols]
conc_df.columns = [i.split('_')[0] for i in conc_df.columns]

# Rename
col_dict = {'SPM':'S.P.M.',
            'TOTN':'n',
            'NH4-N':'nh4',
            'NO3-N':'no3',
            'TOTP':'p',
            'PO4-P':'po4'}
for col in col_dict.keys():
    new_col = col_dict[col]
    lod_df[new_col] = lod_df[col]
    conc_df[new_col] = conc_df[col]
    del conc_df[col], lod_df[col]

# Map Excel headings to df cols
par_dict = {'SPM':'S.P.M.',
            'TOC':'TOC',
            'PO4-P':'po4',
            'P-Total':'p',
            'NO3-N':'no3', 
            'NH4-N':'nh4',
            'N-Total':'n',
            'As':'As',
            'Pb':'Pb',
            'Cd':'Cd', 
            'Cu':'Cu',
            'Zn':'Zn',
            'Ni':'Ni',
            'Total Cr':'Cr',
            'Hg':'Hg'}
    
# Map names to stns
names_dict = {'Orkla':29778,
              'Vefsna':29782,
              'Alta':29779,
              'Glomma':29617,
              'Drammenselva':29612,
              u'Numedalslågen':29615,
              'Skienselva':29613,
              'Otra':29614,
              'Orreelva':29783,
              'Vosso':29821}

# Open new file and get sheet
wb = load_workbook(filename=temp_path)
ws = wb['7']

# Get row numbers
row_dict = {}
for item in ws['B12':'B%s' % ws.max_row]:
    # Get cell properties
    cell = item[0]
    name = cell.value
    row = cell.row
    row_dict[name] = row

# Get col numbers
col_dict = {}
for cell in ws['E9':'AK9'][0]:
    # Get cell properties
    par = cell.value
    col = cell.column
    col_dict[par] = col

pars = ['Ni', 'Pb', 'NH4-N', 'Total Cr', 'NO3-N', 
        'Zn', 'As', 'Cd', 'P-Total', 'SPM', 'PO4-P', 
        'N-Total', 'Hg', 'Cu', 'TOC']

# Update spreadsheet
for reg in names_dict.keys():
    for par in pars:
        # Get values from df
        # 1a. Lower average
        vals = conc_df.loc[names_dict[reg], par_dict[par]].values.copy()
        lods = lod_df.loc[names_dict[reg], par_dict[par]].fillna('0').values.copy()
        vals[(lods=='<')] = 0
        val = np.nanmean(vals)
        
        row = row_dict[reg]
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val
        
        # 1b. Upper average
        val = conc_df.loc[names_dict[reg], par_dict[par]].mean()
        row = row_dict[reg]
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 2. Min
        val = conc_df.loc[names_dict[reg], par_dict[par]].min()
        row = row_dict[reg] + 2
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 3. Max
        val = conc_df.loc[names_dict[reg], par_dict[par]].max()
        row = row_dict[reg] + 3
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val   
        
        # 4. N
        n_samp = len(conc_df.loc[names_dict[reg], par_dict[par]].dropna())
        row = row_dict[reg] + 5
        col = col_dict[par]
        ws.cell(row=row, column=col).value = n_samp 

        # 5. N LOD
        n_lod = (~pd.isnull(lod_df.loc[names_dict[reg], par_dict[par]])).sum()
        pct_lod = 100 * float(n_lod)/float(n_samp)
        
        if pct_lod < 30:
            val = 'Yes'
        else:
            val = 'No'
            
        row = row_dict[reg] + 4
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 6. Std
        val = conc_df.loc[names_dict[reg], par_dict[par]].std()
        row = row_dict[reg] + 7
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val
        
# Save
wb.save(temp_path)

### 2.10. Sheet 9: Discharge

The notebook [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/recalculate_ospar_flows.ipynb) handles the OSPAR flow data. Run this, and then copy the results over to Sheet 9. 